# 2. Injecting Sources

For the Rubin Science Platform at <a href="https://data.lsst.cloud">data.lsst.cloud</a><br>
Data Release: <a href="https://dp1.lsst.io/">Data Preview 1 (DP1)</a> <br>

**Learning Objective:** To inject Rubin images with simulated SL-AGN stamps with the appropriate rotation.

**LSST Data Products:** `visit_image`, `deep_coadd`

**Packages:** `lsst.daf.butler`, `lsst.rsp`, `lsst.afw.display`, `lsst.source.injection`

**Credit:** This notebook was created by Dhruv Sharma with additional code developed by Shenming Fu (SLAC National Accelerator Laboratory)

## 0. Import Packages & Catalog Files

First, import the necessary methods from `lib/` and additional packages.

In [ ]:
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
from lsst.source.injection import generate_injection_catalog
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.table import Table, vstack
from lib.visual import *
from lib.inject import *
from lib.stamp import *

Now let's also import the `visit_image`s we saved last time in the `catalog/` folder using the `load_pickle` method. This way we don't have to query the RSP every time.

In [ ]:
visit_0 = load_pickle("visit_0")
visit_1 = load_pickle("visit_1")
visit_2 = load_pickle("visit_2")
visit_3 = load_pickle("visit_3")
visit_4 = load_pickle("visit_4")
coadd = load_pickle("coadd")

Injection catalogs are essentially tables detailing the sources that are to be injected into any given image. They'll typically contain important info on each source, including its ra,dec or pixel position, stamp filename, magnitude ("brightness"), and more.

## 1. Coadd Images

### 1.1. Coadd Injection Catalog Generation

Let's start by trying to inject our coadd stamps into our `deep_coadd` image. As mentioned last time, we'll do this for 5 copies of the coadd, injecting many instances of only one system into each.

Start by generating injection catalogs for each image we aim to produce using `lib/inject.py`'s method `make_coadd_grid_coord`, which makes a catalogue of stamp injections in a grid pattern. We'll provide the method with our coadd image, the magnitude we want our injected coadd SL-AGN stamp to appear as, and the filename of our stamp.

As a bit of foresight, we aim to inject stamps with varying magnitudes across each image, ranging from 19 to 23. Our coadd stamp should then have the average brightness of all of these, so we'll say mag = 21. Store all the coadd injection catalogs in a list `coadd_inj_catalogs`.

In [ ]:
coadd_inj_catalogs = []
for i in ([1,2,4,5]):
    coadd_inj_catalogs.append(make_coadd_grid_coord(coadd, 21, 'fig/system_%s_coadd.fits'%str(i)))

### 1.2. Coadd Stamp Injection

Now use the `template_inject_stamp` method to inject the stamps from each catalogue into each coadd. The naming `s1_inj_coadd` indicates this is the `deep_coadd` image injected with instances of the coadd stamp representing SL-AGN system 1.

In [ ]:
s1_inj_coadd = template_inject_stamp(coadd, coadd_inj_catalogs[0])
s2_inj_coadd = template_inject_stamp(coadd, coadd_inj_catalogs[1])
s4_inj_coadd = template_inject_stamp(coadd, coadd_inj_catalogs[2])
s5_inj_coadd = template_inject_stamp(coadd, coadd_inj_catalogs[3])

## 2. Visit Images

### 2.1. Visit Injection Catalog Generation

Now generate visit catalogues for each system and time index by constructing numpy Tables containing all the relevant information: each source's injection id, the ra,dec coordinates, the source type (a stamp), the source magnitude, and the stamp filename.

As we provide the ra,dec coordinates of each source, the pixel coordinates are appropriately changed to match each visit's wcs. The magnitude varies from visit to visit, from 19 to 23, and we specify that we want the appropriately rotated stamp for each visit.

In [ ]:
num_sources = len(coadd_inj_catalogs[0])
num_visits = 5
inj_catalogs = []
stamp_systems = [1,2,4,5]
for stamp in range(len(stamp_systems)):
    for visit in range(num_visits):
        inj_catalogs.append(Table())
        for source in range(num_sources):
            inj_catalogs[5*stamp+visit] = vstack([inj_catalogs[5*stamp+visit], Table(
                {'injection_id': [source],
                    'ra': [coadd_inj_catalogs[0][source][0]],
                    'dec': [coadd_inj_catalogs[0][source][1]],
                    'source_type': ['Stamp'],
                    'mag': [19+visit],
                    'stamp': ['fig/rotated_system_%s_%s_wcs.fits'%(stamp_systems[stamp],visit)],
                }
            )])

### 2.2. Visit Stamp Injection

Use the `visit_inject_stamp` method to inject each visit image with the injection catalog stored in its respective entry in inj_catalogs.

In [ ]:
s1_inj_visit_0 = visit_inject_stamp(visit_0, inj_catalogs[0])
s1_inj_visit_1 = visit_inject_stamp(visit_1, inj_catalogs[1])
s1_inj_visit_2 = visit_inject_stamp(visit_2, inj_catalogs[2])
s1_inj_visit_3 = visit_inject_stamp(visit_3, inj_catalogs[3])
s1_inj_visit_4 = visit_inject_stamp(visit_4, inj_catalogs[4])
s2_inj_visit_0 = visit_inject_stamp(visit_0, inj_catalogs[5])
s2_inj_visit_1 = visit_inject_stamp(visit_1, inj_catalogs[6])
s2_inj_visit_2 = visit_inject_stamp(visit_2, inj_catalogs[7])
s2_inj_visit_3 = visit_inject_stamp(visit_3, inj_catalogs[8])
s2_inj_visit_4 = visit_inject_stamp(visit_4, inj_catalogs[9])
s4_inj_visit_0 = visit_inject_stamp(visit_0, inj_catalogs[10])
s4_inj_visit_1 = visit_inject_stamp(visit_1, inj_catalogs[11])
s4_inj_visit_2 = visit_inject_stamp(visit_2, inj_catalogs[12])
s4_inj_visit_3 = visit_inject_stamp(visit_3, inj_catalogs[13])
s4_inj_visit_4 = visit_inject_stamp(visit_4, inj_catalogs[14])
s5_inj_visit_0 = visit_inject_stamp(visit_0, inj_catalogs[15])
s5_inj_visit_1 = visit_inject_stamp(visit_1, inj_catalogs[16])
s5_inj_visit_2 = visit_inject_stamp(visit_2, inj_catalogs[17])
s5_inj_visit_3 = visit_inject_stamp(visit_3, inj_catalogs[18])
s5_inj_visit_4 = visit_inject_stamp(visit_4, inj_catalogs[19])

Run the code below to view the 5 injected `visit_image`s and the injected `deep_coadd` image.

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(10,7), dpi=500)
%matplotlib inline

plt.sca(ax[0][0])
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale') 
display.mtv(s1_inj_visit_0.image)
plt.title('s1_inj_visit_0')
plt.axis('off')

plt.sca(ax[0][1])
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale') 
display.mtv(s1_inj_visit_1.image)
plt.title('s1_inj_visit_1')
plt.axis('off')

plt.sca(ax[0][2])
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale') 
display.mtv(s1_inj_visit_2.image)
plt.title('s1_inj_visit_2')
plt.axis('off')

plt.sca(ax[1][0])
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale') 
display.mtv(s1_inj_visit_3.image)
plt.title('s1_inj_visit_3')
plt.axis('off')

plt.sca(ax[1][1])
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale') 
display.mtv(s1_inj_visit_4.image)
plt.title('s1_inj_visit_4')
plt.axis('off')

plt.sca(ax[1][2])
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale') 
display.mtv(s1_inj_coadd.image)
plt.title('s1_inj_coadd')
plt.axis('off')
plt.show()

## 3. Saving Our Progress

Again use `save_pickle` to save all our new images and catalogs from this notebook to the `catalog/` folder.

In [ ]:
save_pickle("s1_inj_visit_0", s1_inj_visit_0)
save_pickle("s1_inj_visit_1", s1_inj_visit_1)
save_pickle("s1_inj_visit_2", s1_inj_visit_2)
save_pickle("s1_inj_visit_3", s1_inj_visit_3)
save_pickle("s1_inj_visit_4", s1_inj_visit_4)

save_pickle("s2_inj_visit_0", s2_inj_visit_0)
save_pickle("s2_inj_visit_1", s2_inj_visit_1)
save_pickle("s2_inj_visit_2", s2_inj_visit_2)
save_pickle("s2_inj_visit_3", s2_inj_visit_3)
save_pickle("s2_inj_visit_4", s2_inj_visit_4)

save_pickle("s4_inj_visit_0", s4_inj_visit_0)
save_pickle("s4_inj_visit_1", s4_inj_visit_1)
save_pickle("s4_inj_visit_2", s4_inj_visit_2)
save_pickle("s4_inj_visit_3", s4_inj_visit_3)
save_pickle("s4_inj_visit_4", s4_inj_visit_4)

save_pickle("s5_inj_visit_0", s5_inj_visit_0)
save_pickle("s5_inj_visit_1", s5_inj_visit_1)
save_pickle("s5_inj_visit_2", s5_inj_visit_2)
save_pickle("s5_inj_visit_3", s5_inj_visit_3)
save_pickle("s5_inj_visit_4", s5_inj_visit_4)

In [ ]:
save_pickle("s1_inj_coadd", s1_inj_coadd)
save_pickle("s2_inj_coadd", s2_inj_coadd)
save_pickle("s4_inj_coadd", s4_inj_coadd)
save_pickle("s5_inj_coadd", s5_inj_coadd)

Save a copy of the list of injection catalogs we generated, `inj_catalogs` so we can use it in the future.

In [ ]:
save_pickle("inj_catalogs", inj_catalogs)

We'll also save all the data across our injection catalogs as one `astropy.table` for other future use.

In [ ]:
all_inj_sources = vstack([inj_catalogs[0], 
                          inj_catalogs[1], 
                          inj_catalogs[2], 
                          inj_catalogs[3], 
                          inj_catalogs[4],
                          inj_catalogs[5], 
                          inj_catalogs[6], 
                          inj_catalogs[7], 
                          inj_catalogs[8], 
                          inj_catalogs[9],
                          inj_catalogs[10], 
                          inj_catalogs[11], 
                          inj_catalogs[12], 
                          inj_catalogs[13], 
                          inj_catalogs[14],
                          inj_catalogs[15], 
                          inj_catalogs[16], 
                          inj_catalogs[17], 
                          inj_catalogs[18], 
                          inj_catalogs[19],])

save_pickle("all_inj_sources", all_inj_sources)